In [ ]:
import csv

#Semplicemente stampo la lista delle colonne
with open('FerraraFiles/ferrara_airbreak_int_edti_ciclabili_2021.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    list_of_column_names = []
    for row in csv_reader:
        list_of_column_names.append(row)
        break

for i in list_of_column_names:
    print(i)

In [ ]:
#import osm2geojson
import geopandas as gpd

#Creo i dataframe riguardanti i dati di Ferrara, primaquello delle "vie", poi quello delle rastrelliere
#All'interno del wfs inviatomi per mail ci sono i dati riguardanti le strade, però per ora faccio solo la traduzione dei tags

dfWays = gpd.read_file("FerraraFiles/Ciclabili_2021.shp")
for col in dfWays.columns:
    print(col)

dfNodes = gpd.read_file("FerraraFiles/RastrelliereFerrara.shp")
for col in dfNodes.columns:
    print(col)

In [ ]:
dfWays.head()
print(len(dfWays))

In [ ]:
#Prendo gli elementi che hanno come risposta None nelle colonne segnaletic o livello
#Quindi creerò le domande basate su quelle
#Per cominciare faccio solo con queste 2 poi lo sviluppo di èiù
import pandas as pd


my_new_dfWaysSegnaletic = dfWays.loc[(dfWays['segnaletic'].isna())]
my_new_dfWaysLivello = dfWays.loc[(dfWays['livello'].isna())]
print(len(my_new_dfWaysLivello))
print(len(my_new_dfWaysSegnaletic))

my_new_dfWays = [my_new_dfWaysLivello,my_new_dfWaysSegnaletic]
result = pd.concat(my_new_dfWays)
print(len(result))
result.head()

In [ ]:
booleans = result.duplicated(keep=False)
print("booleans series: ")
print(booleans)
print("\n")
result = result[~booleans]

In [ ]:
print(len(result))

In [ ]:
#elimino tutte le colonne tranne geometry, id, gid, livello e segnaletic.
result.drop(result.columns.difference(['gid','id','livello','segnaletic','geometry']), 1, inplace=True)
result.head()

In [ ]:
#Ora creo la lista di domande con tutto associato 
import yaml
from yaml.loader import SafeLoader

yamlFile="create_Ferrara_quest.yaml"
with open(yamlFile) as f:
    data = yaml.load(f,Loader=SafeLoader)
    sorted_data = yaml.dump(data, sort_keys=True)
print(data)

In [157]:
#for index,row in result.iterrows():
    #print(row['segnaletic'])
    #print(row["livello"])


def createQuestion(keyNameDf, my_list, fileName,id,geometry):
    with open(fileName) as f:
        data = yaml.load(f, Loader = SafeLoader)
        for keys,value in data["way-keys"].items():
            keyName = data['way-keys'][keys]["name"]
            keyQuestion = data['way-keys'][keys]["question"]
            keyScore = data['way-keys'][keys]["score"]
            keyValidating = data['way-keys'][keys]["validating"]
            keyAnswers = data['way-keys'][keys]["answers"]
            keyType = "WAY" #FOR NOW
            if(keyNameDf == keyName):
                #print("funzica")
                value = ["WAY", keyQuestion, id,keyName,keyScore,keyValidating,keyAnswers,geometry]
                my_list.append(value)
                return my_list


            

question_list = [["NODE OR WAY", "QUESTION", "ID", "KEY", "SCORE", "VALIDATING","ANSWERS", "shape"]]
for index,row in result.iterrows():
    #print(row[0]) stampa il gid
    #print(row[1]) stampa l'id
    #print(row.index[0])
    #print(row[index])
    #print(index)
    if(row['segnaletic']==None):
        #Crea domanda sulla segnaletic (vedi file) yaml
        question_list = createQuestion("segnaletic", question_list,yamlFile,row['id'],row["geometry"])
    if(row['livello']==None):
        #Crea domanda sul livello (vedi file yaml)
        question_list = createQuestion("livello", question_list,yamlFile,row['id'],row["geometry"])
    #print("STAMPATO LA ROW")
#Per questa prova oin questo momento sono tutte way

#Ora creo il csv con dentro tutte le varie missioni
csvFileName = "FerraraFiles/ForMissionCreation/csvMissionFiles/wayQuestsFerrara.csv"
with open(csvFileName, "w", newline = '')as file:
    writer = csv.writer(file,delimiter = "|")
    writer.writerows(question_list)
    file.close()
#Creo il csv con le varie missioni ma sezna il campo geometria
"""noGeomFileName = "FerraraFiles/ForMissionCreation/csvMissionFiles/wayQuestsFerraraNoGeom.csv"
for i in question_list:
    i.pop()

with open(noGeomFileName, "w", newline = '')as file:
    writer = csv.writer(file,delimiter = "|")
    writer.writerows(question_list)
    file.close()"""

'noGeomFileName = "FerraraFiles/ForMissionCreation/csvMissionFiles/wayQuestsFerraraNoGeom.csv"\nfor i in question_list:\n    i.pop()\n\nwith open(noGeomFileName, "w", newline = \'\')as file:\n    writer = csv.writer(file,delimiter = "|")\n    writer.writerows(question_list)\n    file.close()'

In [1]:
import geopandas
from geopandas import GeoDataFrame

#converto csv in dataframe e quindi geodataframe
csv_df = pd.read_csv(csvFileName,delimiter="|")
geo_df = GeoDataFrame(csv_df)
geo_df.head()

NameError: name 'pd' is not defined

In [ ]:
geo_df.set_geometry('shape', inplace=True)

In [ ]:
from shapely import wkt
shapefile = geo_df
shapefile['shape'] = shapefile['shape'].apply(wkt.loads)
shapefile = geopandas.GeoDataFrame(shapefile, geometry='shape')
shapefile.to_file('FerraraFiles/shapeFiles/MyGeometries.shp', driver='ESRI Shapefile')
#gdf = geopandas.GeoDataFrame.from_file('FerraraFiles/shapeFiles/MyGeometries.shp')
#gdf= gdf.astype({"id": str}, errors='raise') 
#file = "/MyGeometries.shp"

In [ ]:
#Ora elimino la colonna geometry per avere già gli elementi da inserire nel db.
#Anche perchè la conversione delle risposte datedagli user la si farà a fine mapathon
import os
import sqlite3

DB_PATH = os.path.join(os.getcwd(), 'FerraraFiles/database/ferraraTry.db')

In [ ]:
import pandas as pd

my_data = pd.read_csv('FerraraFiles/ForMissionCreation/csvMissionFiles/wayQuestsFerrara.csv', index_col=False, delimiter="|")
my_data = my_data.rename(columns={'NODE OR WAY':'TYPE'})#########
geo_df = geo_df.rename(columns={"NODE OR WAY":"TYPE"})
my_data.drop(my_data.columns.difference(['TYPE','QUESTION','ID','SCORE','VALIDATING','ANSWERS','KEY']), 1, inplace=True)
my_data["ANSWER"] = ""
my_data["USERANSWERED"] = ""
my_data["NUMBEROFVALIDATIONS"] = 0
my_data["USERSWHOVALIDATED"] = ""
print(my_data.head())

with sqlite3.connect(DB_PATH) as conn: 
    df_sql = my_data.to_sql('question_table', conn, if_exists='replace', index = False)

In [ ]:
completed_data = my_data
completed_data.drop(completed_data.columns.difference(["ID","TYPE"]),1,inplace=True)
completed_data["completed"] = "no"
completed_data["TYPE"] = "way"
completed_data = completed_data.rename(columns = {'TYPE':'type', 'ID':'id'})
print(completed_data.head())
with sqlite3.connect(DB_PATH) as conn:
    df_completed = completed_data.to_sql('completed_table',conn,if_exists="replace", index = False)

In [ ]:
with sqlite3.connect(DB_PATH) as conn:
    conn.enable_load_extension(True)
    conn.load_extension("mod_spatialite")
    conn.commit()

#IMPORTANTEEEEEEEEEEEEEEEEE
with sqlite3.connect(DB_PATH) as conn:
    conn.enable_load_extension(True)
    conn.load_extension("mod_spatialite")
    conn.execute("SELECT InitSpatialMetaData(1);")


with sqlite3.connect(DB_PATH) as conn:
    conn.enable_load_extension(True)
    conn.load_extension("mod_spatialite")
    cur = conn.execute(
        """SELECT * FROM 'question_table' WHERE ID IS 1022 AND TYPE IS 'WAY' AND NUMBEROFVALIDATIONS IS NOT 5"""
    )
    resultQuery = cur.fetchall()
print(resultQuery)

#with sqlite3.connect(DB_PATH) as conn:
#    conn.enable_load_extension(True)
#    conn.load_extension("mod_spatialite")
#    cur = conn.execute(
#        """UPDATE completed_table SET completed = "yes" WHERE ID = "884" AND TYPE = 'way'"""
#    )
#    resultQuery = cur.fetchall()

#print(resultQuery)
    
with sqlite3.connect(DB_PATH) as conn:
    conn.enable_load_extension(True)
    conn.load_extension("mod_spatialite")
    cur = conn.execute(
        """SELECT * FROM 'completed_table' WHERE completed IS 'yes'"""
    )
    resultQuery = cur.fetchall()

print(resultQuery)
conn.close()

In [ ]:
#Creo tabella per vedere se id ha già tutte le rispooste o meno
geo_df.drop(geo_df.columns.difference(['ID','TYPE']), 1, inplace=True)
print(geo_df.head())
completed_data = geo_df
completed_data["completed"] = "no"

#print(completed_data.head())
completed_data = gpd.GeoDataFrame(completed_data)

print(len(completed_data))
print(completed_data.head())

#with sqlite3.connect(DB_PATH) as conn:
#    df_completed = completed_data.to_sql('completed_table',conn,if_exists="replace", index = False)

In [ ]:
import json
xmlString = result.to_xml()
#print(xmlString)
with open("FerraraFiles/xmlFiles/provaXml.xml","w") as f:
    f.write(xmlString)

print(len(result))

jsonFile = result.to_json()
with open('map.geojson','w', encoding='utf-8') as f:
    f.write(jsonFile)
    #json.dump(jsonFile,f)

with open('provaformat.json', 'wt') as out:
    res = json.dump(jsonFile, out, sort_keys=True, indent=4, separators=(',', ': '))